In [1]:
# utils
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import ast
import re
import pickle

# evaluation 
from evaluate import load

In [2]:
# open files in generations directory
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [3]:
models_generations = {
    'en': {},
    'it': {},
    'ru': {},
    'ge': {}
}

files = open_files('generations')
for file in files:
    model = re.sub(r"^fine-tuned-|-exp2-gen\d+.csv", "", file)
    gen = re.search(r"gen\d+", file).group()

    model_generations = pd.read_csv('generations/' + file)
    # convert string to list
    model_generations['actual'] = model_generations['actual'].apply(ast.literal_eval)
    
    # if containt -en-
    if '-en-' in file:
        if model not in models_generations['en']:
            models_generations['en'][model] = {}
        models_generations['en'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: en")
    elif '-it-' in file:
        if model not in models_generations['it']:
            models_generations['it'][model] = {}
        models_generations['it'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: it")
    elif '-ge-' in file:
        if model not in models_generations['ge']:
            models_generations['ge'][model] = {}
        models_generations['ge'][model][gen] = model_generations
        print(f"model: {model} - ge: {gen}, lang: ge")
    elif '-ru-' in file:
        if model not in models_generations['ru']:
            models_generations['ru'][model] = {}
        models_generations['ru'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: ru")

model: Llama-3.1-8B-Instruct-en - gen: gen0, lang: en
model: Llama-3.1-8B-Instruct-en - gen: gen1, lang: en
model: Llama-3.1-8B-Instruct-en - gen: gen2, lang: en
model: Llama-3.1-8B-Instruct-ge - ge: gen0, lang: ge
model: Llama-3.1-8B-Instruct-ge - ge: gen1, lang: ge
model: Llama-3.1-8B-Instruct-ge - ge: gen2, lang: ge
model: Llama-3.1-8B-Instruct-it - gen: gen0, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen1, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen2, lang: it
model: Llama-3.1-8B-Instruct-ru - gen: gen0, lang: ru
model: Llama-3.1-8B-Instruct-ru - gen: gen1, lang: ru
model: Llama-3.1-8B-Instruct-ru - gen: gen2, lang: ru
model: Mistral-Nemo-Instruct-2407-en - gen: gen0, lang: en
model: Mistral-Nemo-Instruct-2407-en - gen: gen1, lang: en
model: Mistral-Nemo-Instruct-2407-en - gen: gen2, lang: en
model: Mistral-Nemo-Instruct-2407-ge - ge: gen0, lang: ge
model: Mistral-Nemo-Instruct-2407-ge - ge: gen1, lang: ge
model: Mistral-Nemo-Instruct-2407-ge - ge: gen2, lang: ge
mode

## Valutazione automatica

### <a href="https://huggingface.co/spaces/evaluate-metric/meteor">METEOR</a>

In [4]:
meteor = load('meteor')

Using the latest cached version of the module from C:\Users\OliverioM\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--meteor\e7ed321a1b44c34fa4679192809db2cee7e3bd4bba0fe8b76061d807706c2374 (last modified on Thu Oct 10 16:06:59 2024) since it couldn't be found locally at evaluate-metric--meteor, or remotely on the Hugging Face Hub.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OliverioM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\OliverioM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\OliverioM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
meteor_scores = {'en': {}, 'it': {}, 'ge': {}, 'ru': {}}

for lang in models_generations:
    for model in models_generations[lang]:
        model_meteor_scores = []
        print(f'Language: {lang}, Model: {model}')
        
        for gen in sorted(models_generations[lang][model]):  # Ensure order (gen0, gen1, gen2)
            references = models_generations[lang][model][gen]['actual']
            predictions = models_generations[lang][model][gen]['prediction']

            gen_meteor_scores = []
            for i in tqdm(range(len(references)), desc="Calcolo punteggi METEOR"):
                results = meteor.compute(predictions=[predictions[i]], references=[references[i]])
                gen_meteor_scores.append(results["meteor"])

            model_meteor_scores.append(gen_meteor_scores)  # Store BLEU for each generation
        
        # Store the three METEOR scores instead of their mean
        meteor_scores[lang][model] = model_meteor_scores
        print(f'BLEU scores for {model} ({lang}): {model_meteor_scores}')
        print(f'Average: {np.mean(model_meteor_scores)}')
        print()

Language: en, Model: Llama-3.1-8B-Instruct-en


Calcolo punteggi METEOR: 100%|██████████| 383/383 [00:03<00:00, 104.88it/s]


BLEU scores for Llama-3.1-8B-Instruct-en (en): [[0.9990234375, 0.9353193773483629, 0.5681818181818182, 0.6457115009746588, 0.5365350099079751, 0.8780549256739733, 0.75625, 0.5971471857759483, 0.9990234375, 0.9905133928571428, 0.6755884901104169, 0.7455234159779615, 0.9996243425995492, 0.8979985955056179, 0.8950617283950617, 0.9990234375, 0.9490196078431373, 0.9985422740524781, 0.9283625730994152, 0.9835843169176502, 0.8313775831653225, 0.789243781019589, 0.6782293923155853, 0.9997106481481481, 0.985735843633228, 0.8066666666666668, 0.9997724169321802, 0.6499975701025417, 0.8786823033100167, 0.5353535353535354, 0.8950617283950617, 0.857836285547282, 0.84162772810599, 0.7819174181300428, 0.8300434512725017, 0.9993141289437586, 0.8227909482758621, 0.7089466816235891, 0.9999375, 0.9526488919667591, 0.7883417917431522, 0.889690170940171, 0.7229026760276759, 0.7199882041953272, 0.9366457015045212, 0.7702460567823343, 0.8745874587458746, 0.9995, 0.9985422740524781, 0.8610146186134028, 0.9995,

Calcolo punteggi METEOR: 100%|██████████| 383/383 [00:03<00:00, 98.62it/s] 


BLEU scores for Mistral-Nemo-Instruct-2407-en (en): [[0.9990234375, 0.9353193773483629, 0.5681818181818182, 0.7305194805194806, 0.5513044491031913, 0.9101196803950482, 0.6918799458187781, 0.5336896663427275, 0.9990234375, 0.9905133928571428, 0.8027522935779816, 0.8894491207600632, 0.9996243425995492, 0.738954922628392, 0.8950617283950617, 0.9990234375, 0.9490196078431373, 0.9985422740524781, 0.7258064516129032, 0.9993141289437586, 0.786574074074074, 0.8964143426294822, 0.7773446457234581, 0.9268808114961962, 0.9114583333333334, 0.8066666666666668, 0.9997724169321802, 0.6898601398601399, 0.8816137566137566, 0.5353535353535354, 0.6918367346938775, 0.8140409370652161, 0.7029435016148292, 0.6701150089579841, 0.8300434512725017, 0.9728931830381106, 0.7592654503773045, 0.9537627551020408, 0.9641009852216748, 0.8743523316062176, 0.7958612611440098, 0.889690170940171, 0.5105819397993311, 0.6898846495119786, 0.8416446163240382, 0.8082429582065591, 0.8745874587458746, 0.9799019607843138, 0.99854

...

BLEU scores for Llama-3.1-8B-Instruct-ge (ge): [[0.9985422740524781, 0.9084673899488716, 0.6724489795918367, 0.6048387096774194, 0.3048955463728191, 0.7212094907407408, 0.8057510753589187, 0.4309622368387636, 0.6098360655737705, 0.9998779296875, 0.6676557863501484, 0.920940170940171, 0.9054545454545455, 0.8979985955056179, 0.9993141289437586, 0.5563218390804598, 0.5318681318681319, 0.9997106481481481, 0.860948667966212, 0.854119425547997, 0.876565060635857, 0.9438134228050196, 0.6490131578947369, 0.8103975535168196, 0.9733514001806685, 0.9976851851851852, 0.9997106481481481, 0.4508928571428571, 0.7987079037800686, 0.48478260869565226, 0.618131868131868, 0.7670454545454546, 0.8389171075837741, 0.6489948311237375, 0.6713681849551414, 0.9993141289437586, 0.7051990346311535, 0.9438513106090165, 0.9999460101500918, 0.9402268760907505, 0.5258550572795324, 0.9259259259259259, 0.3880070546737213, 0.4920764867551605, 0.8810643991712706, 0.8039867109634551, 0.855940934065934, 0.8775114984265312,

Calcolo punteggi METEOR: 100%|██████████| 383/383 [00:03<00:00, 103.05it/s]


BLEU scores for Mistral-Nemo-Instruct-2407-ge (ge): [[0.9985422740524781, 0.9084673899488716, 0.42216721672167207, 0.6048387096774194, 0.4838423295454545, 0.8141321044546851, 0.8270058369608231, 0.43835978835978845, 0.43314500941619577, 0.9998779296875, 0.726739537646199, 0.8186789287267755, 0.6918367346938775, 0.6691919191919192, 0.9993141289437586, 0.5563218390804598, 0.5318681318681319, 0.9498207885304659, 0.9213952850877193, 0.9995, 0.7932930086680797, 0.9349145063430778, 0.6410596026490065, 0.8030303030303031, 0.8151159690749132, 0.8066666666666668, 0.9997106481481481, 0.6136363636363636, 0.643598615916955, 0.6400966183574879, 0.618131868131868, 0.798826530612245, 0.8522727272727272, 0.5600841869314777, 0.7725591715976331, 0.9728931830381106, 0.7496253576131874, 0.7816606293060586, 0.9999460101500918, 0.9467186484730344, 0.577453276728639, 0.889690170940171, 0.4993288590604027, 0.6639462775381941, 0.8147445157389055, 0.5957054539450753, 0.7471655328798186, 0.6861724281549355, 0.99

Calcolo punteggi METEOR: 100%|██████████| 383/383 [00:03<00:00, 104.62it/s]

BLEU scores for Qwen2.5-7B-Instruct-ge (ge): [[0.9985422740524781, 0.9995, 0.372, 0.6104651162790697, 0.5828578838174273, 0.7186411149825784, 0.6753535353535353, 0.44002112629563606, 0.6098360655737705, 0.9259259259259259, 0.5813517639852629, 0.8248810418231906, 0.9861386138613862, 0.8070175438596492, 0.6545430672268908, 0.3703703703703703, 0.6400966183574879, 0.8493645392041114, 0.6944444444444445, 0.9995, 0.7720588235294117, 0.7324630386914124, 0.7016690973910226, 0.6756756756756757, 0.6229323308270678, 0.9976851851851852, 0.9997106481481481, 0.4479817757361248, 0.7036124567474049, 0.6400966183574879, 0.66167290886392, 0.8130612244897959, 0.8090413716507011, 0.5495925320564471, 0.8848012889366272, 0.7687074829931974, 0.6463952282157674, 0.6553603516640182, 0.9999460101500918, 0.9941842757900539, 0.7526014358312495, 0.9641356881521297, 0.4905987933062648, 0.46073264986745877, 0.8810643991712706, 0.6571162667021205, 0.7471655328798186, 0.7425742574257427, 0.9985422740524781, 0.74957983

In [ ]:
# Save meteor scores
with open('meteor_scores-exp2.pkl', 'wb') as f:
    pickle.dump(meteor_scores, f)